In [ ]:
import scipy.io
mat = scipy.io.loadmat(r'D:\fUSI\Code base\test_data\test\fr1.mat')
print(mat.keys())
print(mat['x'].shape)
print(mat['y'].shape)

In [ ]:
import torch
from torch.utils.data import Dataset
import scipy.io
import numpy as np
import os
from glob import glob

class fUSI(Dataset):
    def __init__(self, folder_path, mat_key='y', transform=None):
        self.mat_files = sorted(glob(os.path.join(folder_path, '*.mat')))
        self.mat_key = mat_key
        self.transform = transform

    def __len__(self):
        return len(self.mat_files)

    def __getitem__(self, idx):
        mat_file = self.mat_files[idx]
        data = scipy.io.loadmat(mat_file)
        img = data[self.mat_key].astype(np.float32)  # e.g. key = 'y'

        # Add channel dimension if needed
        if img.ndim == 2:
            img = np.expand_dims(img, axis=0)  # (1, H, W)
        elif img.ndim == 3 and img.shape[-1] <= 3:
            img = img.transpose(2, 0, 1)  # (C, H, W)

        img_tensor = torch.tensor(img)

        if self.transform:
            img_tensor = self.transform(img_tensor)

        return img_tensor


In [ ]:
from torch.utils.data import DataLoader
folder_path = r'D:\fUSI\Code base\test_data\test'
fUSI_data = fUSI(folder_path, mat_key='y')
dataloader = DataLoader(fUSI_data, batch_size=4, shuffle=True)
for batch in dataloader:
    print(batch.shape)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class KSparseAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, k_sparsity):
        super(KSparseAutoencoder, self).__init__()

        # Encoder: Fully connected layers
        self.encoder = nn.Linear(input_dim, hidden_dim)
        
        # Decoder: Fully connected layers
        self.decoder = nn.Linear(hidden_dim, input_dim)
        
        self.k_sparsity = k_sparsity  # sparsity level

    def forward(self, x):
        # Encoding: Compute sparse code
        sparse_code = self.encoder(x)
        
        # Apply sparsity
        sparse_code = self.apply_sparsity(sparse_code)
        
        # Decoding: Reconstruct input
        reconstructed = self.decoder(sparse_code)
        
        return reconstructed, sparse_code

    def apply_sparsity(self, sparse_code):
        # Apply k-sparsity (set all but top k elements to 0)
        topk_values, topk_indices = torch.topk(sparse_code.abs(), self.k_sparsity, dim=-1)
        sparse_code = torch.zeros_like(sparse_code)
        sparse_code.scatter_(1, topk_indices, topk_values)
        return sparse_code


In [ ]:
def reconstruction_loss(original, reconstructed):
    return nn.MSELoss()(reconstructed, original)

def sparsity_loss(sparse_code, k_sparsity):
    # Penalty for sparsity violation (encourages k-sparsity)
    return torch.sum(torch.abs(sparse_code)) - k_sparsity


In [ ]:
def train_k_sparse_autoencoder(model, dataloader, num_epochs=10, lr=0.001):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    for epoch in range(num_epochs):
        total_loss = 0
        for data in dataloader:
            optimizer.zero_grad()
            
            # Flatten the image data (assuming data is in shape [B, C, H, W])
            data_flat = data.view(data.size(0), -1)  # B x (C * H * W)
            
            # Forward pass: get reconstructed output and sparse code
            reconstructed, sparse_code = model(data_flat)
            
            # Compute reconstruction loss
            rec_loss = reconstruction_loss(data_flat, reconstructed)
            
            # Compute sparsity loss
            sparsity_penalty = sparsity_loss(sparse_code, model.k_sparsity)
            
            # Total loss (combine both)
            loss = rec_loss + sparsity_penalty
            
            # Backward pass
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(dataloader)}")
        visualize_dictionary(model)


In [ ]:
# Example: Set dimensions for the model
input_dim = 96 * 96  # assuming input images are 64x64
hidden_dim = 512     # size of the hidden sparse code
k_sparsity = 10      # top-k non-zero elements in the sparse code

# Initialize model
model = KSparseAutoencoder(input_dim, hidden_dim, k_sparsity)

# Assuming you already have a DataLoader for your dataset
# Train the model
train_k_sparse_autoencoder(model, dataloader, num_epochs=10)


In [ ]:
import matplotlib.pyplot as plt
def visualize_dictionary(model):
    # Extract the weights of the encoder layer (first layer)
    dictionary = model.encoder.weight.data.cpu().numpy()

    # Reshape to display each learned feature (assuming each feature is a flattened image)
    num_features = dictionary.shape[0]
    feature_size = int(np.sqrt(dictionary.shape[1]))  # assuming square images, adjust if needed
    dictionary_reshaped = dictionary.reshape(num_features, feature_size, feature_size)

    # Plot learned dictionary (each feature)
    num_cols = 8
    num_rows = num_features // num_cols
    fig, axs = plt.subplots(num_rows, num_cols, figsize=(num_cols * 1.5, num_rows * 1.5))

    for i in range(num_features):
        ax = axs[i // num_cols, i % num_cols]
        ax.imshow(dictionary_reshaped[i], cmap='gray')
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()


In [ ]:
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from overcomplete.sae import TopKSAE, train_sae

# Define your activation tensor (ensure N and d are set)
N, d = 40, 9216  # Example with 10,000 samples of 784-dim activations (e.g., MNIST flattened)
Activations = torch.randn(N, d).to('cpu')

# Initialize Sparse Autoencoder
sae = TopKSAE(d, nb_concepts=16_000, top_k=10, device='cpu')

# Optimizer
optimizer = torch.optim.Adam(sae.parameters(), lr=5e-4)

# Loss function
def criterion(x, x_hat, pre_codes, codes, dictionary):
    mse = (x - x_hat).square().mean()
    return mse

# Training the model
logs = train_sae(sae, dataloader, criterion, optimizer, nb_epochs=20, device='cpu')

# Visualizing the learned dictionary
def visualize_dictionary(sae, num_atoms=10):
    dictionary = sae.encoder.weight.data.cpu().numpy()  # Extract learned dictionary
    fig, axes = plt.subplots(1, num_atoms, figsize=(num_atoms * 2, 2))
    
    for i in range(num_atoms):
        ax = axes[i]
        ax.imshow(dictionary[i].reshape(28, 28), cmap='gray')  # Assuming 28x28 for MNIST
        ax.axis('off')
    
    plt.show()

# After training, visualize the learned dictionary
visualize_dictionary(sae)


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import scipy.io
import numpy as np
import os
from glob import glob
import matplotlib.pyplot as plt

from overcomplete.sae import TopKSAE, train_sae

# Dataset
class fUSI(Dataset):
    def __init__(self, folder_path, mat_key='y', transform=None):
        self.mat_files = sorted(glob(os.path.join(folder_path, '*.mat')))
        self.mat_key = mat_key
        self.transform = transform

    def __len__(self):
        return len(self.mat_files)

    def __getitem__(self, idx):
        mat_file = self.mat_files[idx]
        data = scipy.io.loadmat(mat_file)
        img = data[self.mat_key].astype(np.float32)

        if img.ndim == 2:
            img = np.expand_dims(img, axis=0)  # (1, H, W)
        elif img.ndim == 3 and img.shape[-1] <= 3:
            img = img.transpose(2, 0, 1)  # (C, H, W)

        img_tensor = torch.tensor(img)

        # 🔧 Fix: Use reshape instead of view for non-contiguous tensor
        img_tensor = img_tensor.reshape(-1)

        if self.transform:
            img_tensor = self.transform(img_tensor)

        return img_tensor

# Initialize Dataset and Dataloader
folder_path = r"D:\fUSI\Code base\test_data\test"  # ⬅️ Update this path
fUSI_data = fUSI(folder_path, mat_key='y')
dataloader = DataLoader(fUSI_data, batch_size=1, shuffle=True)

# Set dimensions
sample = next(iter(dataloader))
input_dim = sample.shape[1]  # should be H * W

# Define SAE model
sae = TopKSAE(input_dim, nb_concepts=16000, top_k=10, device='cpu')
optimizer = torch.optim.Adam(sae.parameters(), lr=5e-4)

# Define loss function
def criterion(x, x_hat, pre_codes, codes, dictionary):
    return (x - x_hat).square().mean()

# Train the model
logs = train_sae(sae, dataloader, criterion, optimizer, nb_epochs=20, device='cpu')

In [ ]:
# Visualize the learned dictionary
def visualize_dictionary(sae, num_atoms, img_size=(96, 96)):
    dictionary = sae.get_dictionary().detach().cpu().numpy()  # Shape: (nb_concepts, input_dim)
    
    plt.figure(figsize=(15, 3))
    for i in range(num_atoms):
        atom = dictionary[i].reshape(img_size)
        plt.subplot(1, num_atoms, i + 1)
        plt.imshow(atom, cmap='gray')
        plt.axis('off')
    plt.suptitle("TopKSAE Learned Dictionary Atoms")
    plt.show()

# Show dictionary atoms
visualize_dictionary(sae, num_atoms=10, img_size=(96, 96)) 

In [ ]:
import matplotlib.pyplot as plt

# Get one sample from the dataset
sample_input = next(iter(dataloader))  # shape: (batch_size, input_dim)
sample_input = sample_input[0].unsqueeze(0)  # select first image in batch, keep batch dim

# Move to device
sample_input = sample_input.to(sae.device)

# Forward pass through SAE
with torch.no_grad():
    z_pre, z, x_hat = sae(sample_input)  # x_hat is the reconstructed input

# Convert tensors to numpy for visualization
original = sample_input.cpu().numpy().reshape(1, 96, 96)  # assuming original shape was (1, H, W)
reconstructed = x_hat.cpu().numpy().reshape(1, 96, 96)

# Plot
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.imshow(original[0], cmap='gray')
plt.title("Original Image")

plt.subplot(1, 2, 2)
plt.imshow(reconstructed[0], cmap='gray')
plt.title("Reconstructed Image")
plt.show()


In [ ]:
torch.save(sae.state_dict(), "trained_sae_model.pth")